In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import datetime
from scipy import stats
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 14
fig_size[1] = 10

First we retrieve relevant dispatcher events using Dune. Check out the SQL queries here: 
- Worker **Commitments** https://dune.xyz/queries/48938
This pulls, on each period, each (staker) address – whose correspondng worker successfully committed to the next period. 
- Staker **Locks** https://dune.xyz/queries/54332
This pulls, on each period, any token locking event by a staker and the sum of tokens locked.
- Staker **Withdrawals** https://dune.xyz/queries/54339
This pulls, on each period, any token withdrawal event by a staker and the sum of tokens withdrawn. 

Note that the Locks and Withdrawals tables do not include preallocated holdings or the growth in stakes due to restaking, and so underestimate the total capital staked at a given period – hence this notebook is WIP. For now, this study provides some insight into the variance of committed capital period-to-period, how this impacts the fluctuations of the break-even stake size, and how this in turn affects the population of nodes (addresses) and their respective solvencies. 

In [2]:
df_c = pd.read_csv(r'commitments.csv')
df_l = pd.read_csv(r'locks.csv')
df_w = pd.read_csv(r'withdrawals.csv')
df_c['timestamp'] = pd.to_datetime(df_c['timestamp']) #standardize the date format
df_l['timestamp'] = pd.to_datetime(df_l['timestamp'])
df_w['timestamp'] = pd.to_datetime(df_w['timestamp'])
df_c['timestamp'] = df_c['timestamp'].dt.date
df_l['timestamp'] = df_l['timestamp'].dt.date
df_w['timestamp'] = df_w['timestamp'].dt.date

In [4]:
# Join the three tables into one 
df_temp = pd.merge(df_c, df_l, how='left', on=['Staker', 'timestamp']) 
df_merged = pd.merge(df_temp, df_w, how='left', on=['Staker', 'timestamp'])
df_merged.fillna(0, inplace=True)

In [ ]:
#We focus on node behavior up until the period duration extension on April 14th 2021 (i.e. the era when periods were 1 day long)
split_date = pd.to_datetime('2021-04-15')
df_1day = df_merged.loc[(df_merged['timestamp'] < split_date)]
df_7day = df_merged.loc[(df_merged['timestamp'] >= split_date)]

In [ ]:
#We calculate a rudimentary value for each staker's 
df_1day['Staked'] = df_1day['Locked'].copy() - df_1day['Withdrawn'].copy()
